## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Esperance,AU,-33.87,121.90,60.80,87,7,14.99,clear sky
1,1,Caravelas,BR,-17.71,-39.25,73.71,87,3,6.91,clear sky
2,2,Ushuaia,AR,-54.80,-68.30,30.20,86,90,32.21,heavy snow
3,3,Rikitea,PF,-23.12,-134.97,72.07,82,85,25.41,overcast clouds
4,4,Caborca,MX,30.58,-111.10,90.19,14,0,4.36,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Caravelas,BR,-17.71,-39.25,73.71,87,3,6.91,clear sky
3,3,Rikitea,PF,-23.12,-134.97,72.07,82,85,25.41,overcast clouds
5,5,Bintulu,MY,3.17,113.03,80.60,88,20,4.70,few clouds
8,8,Lagoa,PT,39.05,-27.98,69.33,79,5,12.48,clear sky
10,10,Kapaa,US,22.08,-159.32,86.00,62,20,8.05,few clouds
11,11,Chabahar,IR,25.29,60.64,77.00,83,0,2.89,mist
13,13,Hilo,US,19.73,-155.09,86.00,66,20,8.05,few clouds
17,17,Ribeira Grande,PT,38.52,-28.70,69.40,81,19,12.19,few clouds
19,19,Lompoc,US,34.64,-120.46,73.40,77,1,14.99,clear sky
21,21,Carnarvon,AU,-24.87,113.63,68.00,77,0,17.22,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                336
City                   336
Country                336
Lat                    336
Lng                    336
Max Temp               336
Humidity               336
Cloudiness             336
Wind Speed             336
Current Description    336
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Caravelas,BR,73.71,clear sky,-17.71,-39.25,
3,Rikitea,PF,72.07,overcast clouds,-23.12,-134.97,
5,Bintulu,MY,80.60,few clouds,3.17,113.03,
8,Lagoa,PT,69.33,clear sky,39.05,-27.98,
10,Kapaa,US,86.00,few clouds,22.08,-159.32,
11,Chabahar,IR,77.00,mist,25.29,60.64,
13,Hilo,US,86.00,few clouds,19.73,-155.09,
17,Ribeira Grande,PT,69.40,few clouds,38.52,-28.70,
19,Lompoc,US,73.40,clear sky,34.64,-120.46,
21,Carnarvon,AU,68.00,clear sky,-24.87,113.63,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df=hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(25.0, 101.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))